In [1]:
import pandas as pd
import numpy as np

# Función para calcular esperanza de vida a partir de una tabla de mortalidad
def calcular_esperanza_vida(df, columna_qx):
    edades = df['edad'].values
    qx = df[columna_qx].values
    n = len(edades)

    # Asegurar que qx esté en proporción
    qx = np.where(qx > 1, qx / 100, qx)
    lx = np.zeros(n)
    dx = np.zeros(n)
    Lx = np.zeros(n)
    Tx = np.zeros(n)
    ex = np.zeros(n)

    lx[0] = 100000  # base de la tabla

    for i in range(n - 1):
        dx[i] = lx[i] * qx[i]
        lx[i + 1] = lx[i] - dx[i]
        Lx[i] = lx[i] - dx[i] / 2
        Lx[-1] = lx[-1] / 2
        Tx[-1] = Lx[-1]

    for i in reversed(range(n - 1)):
        Tx[i] = Lx[i] + Tx[i + 1]
        ex = Tx / lx
    return ex

# Edades objetivo
edades_objetivo = [15, 25, 35, 45, 55, 65]

# Leer archivo
archivo = "Tablas CSO.xlsx"
hojas = pd.read_excel(archivo, sheet_name=["hombres", "mujeres"])

# Procesar hoja
def procesar_hoja(df):
    df = df.copy()
    df_resultado = pd.DataFrame()
    df_resultado["edad"] = df["edad"]
    for anio in [1980, 2001, 2017]:
        df_resultado[f"e(x)_CSO{anio}"] = calcular_esperanza_vida(df, anio)
    return df_resultado[df_resultado["edad"].isin(edades_objetivo)]

# Calcular esperanzas
esperanza_hombres = procesar_hoja(hojas["hombres"])
esperanza_mujeres = procesar_hoja(hojas["mujeres"])
